In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('train_engineered.csv')
test = pd.read_csv('test_engineered.csv')

In [37]:
test.shape

(4898, 21009)

In [3]:
train.isnull().sum()

Unnamed: 0               0
Subject                115
Body                     0
Label                    0
SenderEmailAddress       0
url                      0
clean_esc_body           0
Domain_from              1
clean_subject            0
sub_pref_label_0         0
sub_pref_label_1         0
sub_pref_label_2         0
sub_pref_label_3         0
symbol_score             0
new_url                  0
num_url                  0
ppn_count                0
text_word_count          0
stop_word_list           0
body_pct_stop_words      0
domain_cont_url          0
dtype: int64

In [4]:
null_from_index_sub = train.loc[train['clean_subject'].isnull() == True].index
for i in null_from_index_sub:
  train.loc[i,'clean_subject']="NOSUB"

null_domain = train.loc[train['Domain_from'].isnull()==True].index
for i in null_domain:
  train.loc[i,'Domain_from']="NODOM"

null_from_index_body = train.loc[train['clean_esc_body'].isnull() == True].index
for i in null_from_index_body:
  train.loc[i,'clean_esc_body']="NOBOD"

In [5]:
X = train[['clean_subject', 'clean_esc_body', 'Domain_from', 'new_url', 'sub_pref_label_0', 'sub_pref_label_1',
           'sub_pref_label_2', 'sub_pref_label_3', 'num_url', 'symbol_score', 'ppn_count', 'domain_cont_url',
          'body_pct_stop_words']]
y = train['Label']

In [6]:
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

In [7]:
vectorizer_sub = TfidfVectorizer(ngram_range = (1, 2), max_features = 8000)
vectorizer_bod = TfidfVectorizer(ngram_range = (1, 3), max_features = 12000)
vectorizer_dom = TfidfVectorizer(max_features = 500)
vectorizer_url = TfidfVectorizer(max_features = 500)

# tfidf_vectorizer = TfidfVectorizer(ngram_range = (1, 2), max_features = 10000)
# tfidf_sub = tfidf_vectorizer.fit_transform(X_train_sub)
# tfidf_bod = tfidf_vectorizer.fit_transform(X_train_bod)

column_transformer = ColumnTransformer(
    [('tfidf_sub', vectorizer_sub, 'clean_subject'), 
    ('tfidf_bod', vectorizer_bod, 'clean_esc_body'),
    ('tfidf_dom', vectorizer_dom, 'Domain_from'),
    ('tfidf_url', vectorizer_url, 'new_url')],
    remainder='passthrough')

column_transformer.fit(X_train_sub)

ColumnTransformer(remainder='passthrough',
                  transformers=[('tfidf_sub',
                                 TfidfVectorizer(max_features=8000,
                                                 ngram_range=(1, 2)),
                                 'clean_subject'),
                                ('tfidf_bod',
                                 TfidfVectorizer(max_features=12000,
                                                 ngram_range=(1, 3)),
                                 'clean_esc_body'),
                                ('tfidf_dom', TfidfVectorizer(max_features=500),
                                 'Domain_from'),
                                ('tfidf_url', TfidfVectorizer(max_features=500),
                                 'new_url')])

In [8]:
X_train_sub = column_transformer.transform(X_train_sub)
X_test_sub = column_transformer.transform(X_test_sub)

In [9]:
X_test_sub.shape

(1199, 21009)

In [10]:
X_train_sub.shape

(4794, 21009)

In [11]:
xgb_clf = XGBClassifier(random_state = 42)
param_grid = {'n_estimators':[80, 90, 100, 110, 120], 'max_depth':[2, 3, 4, 6],
              'learning_rate':[0.01, 0.1, 0.001, 0.2, 0.3], 'alpha':[0.01, 0.1, 1, 10]}
gridsearch = GridSearchCV(xgb_clf, param_grid = param_grid, cv = 10, verbose = 2).fit(X_train_sub, y_train_sub)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits
[15:50:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] END alpha=0.01, learning_rate=0.01, max_depth=2, n_estimators=80; total time=   1.2s
[15:50:43] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] END alpha=0.01, learning_rate=0.01, max_depth=2, n_estimators=80; total time=   1.1s
[15:50:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] END alpha=10, learning_rate=0.3, max_depth=6, n_estimators=120; total time=   1.3s
[17:57:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] END alpha=10, learning_rate=0.3, max_depth=6, n_estimators=120; total time=   1.3s
[17:57:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [15]:
gridsearch.best_params_

{'alpha': 0.1, 'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 100}

In [16]:
gridsearch.best_score_

0.9891527487821851

In [17]:
model = XGBClassifier(alpha = 0.1, learning_rate = 0.3, max_depth = 3, n_estimators = 100).fit(X_train_sub, y_train_sub)
predictions = model.predict(X_test_sub)
f1_score(y_test_sub, predictions)

[18:10:26] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9865546218487395

In [18]:
test.isnull().sum()

Unnamed: 0              0
SenderEmailAddress      1
Subject                 7
Body                    0
url                     0
clean_esc_body          0
Domain_from            13
clean_subject           0
sub_pref_label_0        0
sub_pref_label_1        0
sub_pref_label_2        0
sub_pref_label_3        0
symbol_score            0
new_url                 0
num_url                 0
ppn_count               0
text_word_count         0
stop_word_list          0
body_pct_stop_words    12
domain_cont_url         0
dtype: int64

In [19]:
null_from_index_test_sub = test.loc[test['clean_subject'].isnull() == True].index
for i in null_from_index_test_sub:
  test.loc[i,'clean_subject']="NOSUB"

null_domain_test = test.loc[test['Domain_from'].isnull()==True].index
for i in null_domain_test:
  test.loc[i,'Domain_from']="NODOM"

null_from_index_test_body = test.loc[test['clean_esc_body'].isnull() == True].index
for i in null_from_index_test_body:
  test.loc[i,'clean_esc_body']="NOBOD"

test['body_pct_stop_words'] = test['body_pct_stop_words'].fillna(0)

In [20]:
test.isnull().sum()

Unnamed: 0             0
SenderEmailAddress     1
Subject                7
Body                   0
url                    0
clean_esc_body         0
Domain_from            0
clean_subject          0
sub_pref_label_0       0
sub_pref_label_1       0
sub_pref_label_2       0
sub_pref_label_3       0
symbol_score           0
new_url                0
num_url                0
ppn_count              0
text_word_count        0
stop_word_list         0
body_pct_stop_words    0
domain_cont_url        0
dtype: int64

In [21]:
test = test[['clean_subject', 'clean_esc_body', 'Domain_from', 'new_url', 'sub_pref_label_0', 'sub_pref_label_1',
           'sub_pref_label_2', 'sub_pref_label_3', 'num_url', 'symbol_score', 'ppn_count', 'domain_cont_url',
          'body_pct_stop_words']]

test = column_transformer.transform(test)

In [22]:
test.shape

(4898, 21009)

In [23]:
predictions_test = model.predict(test)

In [28]:
predictions_test[110]

1

In [27]:
predictions_test.sum()

915

In [29]:
predictions_proba = model.predict_proba(test)

In [30]:
predictions_proba

array([[9.99919593e-01, 8.03786679e-05],
       [5.99893808e-01, 4.00106221e-01],
       [9.93915975e-01, 6.08404120e-03],
       ...,
       [9.99710321e-01, 2.89649528e-04],
       [9.87624407e-01, 1.23755755e-02],
       [9.95364487e-01, 4.63549886e-03]], dtype=float32)

In [31]:
flag = predictions_proba[:, 1]
flag

array([8.03786679e-05, 4.00106221e-01, 6.08404120e-03, ...,
       2.89649528e-04, 1.23755755e-02, 4.63549886e-03], dtype=float32)

In [32]:
ccac_test = pd.read_excel('CCAC_data.xlsx')
Id = ccac_test['Id']
submission = pd.DataFrame({'Id':Id, 'Flag':flag})
submission.to_csv('preprocessed_features_xgb.csv')

In [39]:
submission.set_index('Id', inplace = True)
submission.head()

,Flag
Id,
1,0.000080
2,0.400106
3,0.006084
4,0.006528
5,0.073773


In [40]:
submission.to_csv('preprocessed_features_xgb.csv')

In [41]:
model.feature_importances_[21000:]

array([0.00150345, 0.        , 0.        , 0.        , 0.03254538,
       0.        , 0.0050328 , 0.01597801, 0.00150104], dtype=float32)